<a href="https://colab.research.google.com/github/SufyAD/AI-ML/blob/main/agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -qU langchain_community langchain_text_splitters langchain_google_genai langchain_huggingface langchain_core langchainhub langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00


In [12]:
!pip install -qU wikipedia arxiv faiss-cpu sentence_transformers

## Tool 1: Create `Wikipedia Search Tool`

In [9]:
# Tool 1
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

api_wrapper = WikipediaAPIWrapper(top_k_results=1, lang='en', max_chars=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool.name

'wikipedia'

## Tool 2: Create `WebBase Loader`

In [59]:
# Tool 2
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# These are the steps to store embedded-data in the vector db
#Step1: data ingestion
loader = WebBaseLoader("https://docs.smith.langchain.com")
data = loader.load()

# Step2: Character text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data) # text splitted docs

# Step3: Store in Vector DB, FAISS in our case
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
db = FAISS.from_documents(docs[:100], embeddings)

# Step4: Create retriever from vector DB
web_tool = db.as_retriever()

In [ ]:
# note .invoke == document_get_similar in the new API
web_tool.invoke("What is Extended Neural GPU?")

In [ ]:
### Creating WebTool Wrapper as existing tool is not compatible with OpenAI function calling format
# from langchain.agents import Tool
# def run_web_tool(query: str) -> str:
#     """Searches the vector database for relevant information."""
#     return web_tool.get_relevant_documents(query)[0].page_content # getting retrieved response from vectordb

# web_tool_langchain = Tool(
#     name = "web_base_search",
#     func=run_web_tool,
#     description="Useful for when you need to answer questions about current events. You should ask targeted questions"
# )
# web_tool_langchain


###----- Alternatively, we can use retrieval tool from langchain
from langchain.tools.retriever import create_retriever_tool
web_tool_langchain = create_retriever_tool(
    retriever=web_tool,
    name="web_base_search",
    description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)
web_tool_langchain.invoke("LangSmith is framework-agnostic?")

## Tool 3: Create `Arxiv Research-paper Searcher`

In [ ]:
# Tool 3
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=100)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

# Step 5: Create `OpenAI Tool-Agent` including the LLM(binded with tools)
*   I am using `create_openai_tools_agent` to create agent
*   To run this agent we need `agent_executor` that'll be able to understand the context and pass the query to respective tool



In [ ]:
# Creating a list of tools to be used the LLM
tools = [wiki_tool, web_tool_langchain, arxiv_tool]
tools

In [65]:
### ----------- Calling predefined prompts in the Langchain Hub
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [6]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

gemini = ChatGoogleGenerativeAI(google_api_key=userdata.get('GEMINI_API_KEY'), model="gemini-1.5-flash")

In [ ]:
from langchain.agents import create_openai_tools_agent, Tool, AgentExecutor
from langchain.agents import AgentExecutor
# 1. Create the agent
agent = create_openai_tools_agent(llm=gemini, tools=tools, prompt=prompt)
agent

In [67]:
# 2. Create agent_executor, This is our final agent_executor that will use the tools depending on the context
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({
    "input": "LangSmith allows me to closely monitor my applications? Is this right?"
})